In [1]:
import os
import io
import hashlib
import pickle
import subprocess
import numpy as np
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

WORK_DIR = "/Users/oliver/Documents/p5control-bluefors-evaluation/theory/carlosfcs/"
CACHE_DIR = os.path.join(WORK_DIR, "./.cache")
os.makedirs(CACHE_DIR, exist_ok=True)

In [14]:
V = 83e-6
dV = 1e-9
temperature_K: float = 0.0
energy_gap_V: float = 2e-4
dynes_parameter_V: float = 0.0
transmission: float = 0.5

nmax: int = 10
iw: int = 2003
nchi: int = 66

workdir = "/Users/oliver/Documents/p5control-bluefors-evaluation/theory/carlosfcs/"
fcs_exe = os.path.join(workdir, "fcs")
fcs_in = os.path.join(workdir, f"fcs.in")
tmp_input_dir = os.path.join(workdir, ".tmp")
os.makedirs(tmp_input_dir, exist_ok=True)

tmp_in = os.path.join(
    tmp_input_dir,
    f"[{V} {dV}].in",
)

if dynes_parameter_V <= 0:
    dynes_parameter_V = 1e-7

with open(fcs_in, "r") as f:
    lines = f.readlines()

lines[0] = f"{transmission:.5f} (transmission)\n"
lines[1] = f"{temperature_K:.5f} (temp in K)\n"
lines[2] = f"{energy_gap_V*1e3:.6f} {energy_gap_V*1e3:.6f} (gap1,gap2 in meV)\n"
lines[3] = (
    f"{dynes_parameter_V*1e3:.6f} {dynes_parameter_V*1e3:.6f} (eta1,eta2 = broadening in meV)\n"
)
lines[4] = (
    f"{V*1e3:.8f}, {V*1e3:.8f}, 1.0 (v in mV)\n"
)
lines[5] = f"{nmax} {iw} {nchi} (nmax,iw,nchi)\n"

with open(tmp_in, "w") as f:
    f.writelines(lines)

proc = subprocess.run(
    [fcs_exe],
    stdin=open(tmp_in, "r"),
    capture_output=True,
    text=True,
    cwd=workdir,
)
np.genfromtxt(io.StringIO(proc.stdout), dtype="float64")

array([0.083     , 0.27577558, 0.        , 0.        , 0.00072827,
       0.001411  , 0.01661314, 0.16766054, 0.07531005, 0.01281065,
       0.00124192, 0.        ])